# Outline

- GMM
    - kmeans + gmm(https://ipvs.informatik.uni-stuttgart.de/mlr/marc/teaching/19-MachineLearning/06-unsupervised.pdf)
    - kmeans implementation (https://www.kaggle.com/andyxie/k-means-clustering-implementation-in-python)
- HMM
    - hmm theoretical (https://ipvs.informatik.uni-stuttgart.de/mlr/marc/teaching/18-ArtificialIntelligence/09-graphicalModels.pdf)
    - hmm implementation heads + tails (https://github.com/ananthpn/pyhmm)
    - cont time and discrete time plots (https://github.com/lopatovsky/HMMs)
    - hmm robo filtering (https://github.com/beneisnr/hMm-filtering)
- DTW

- pbdlib
    - Related Work:
    - no symbolic approach (look up "Natural methods for robot task learning: Instructive demonstrations, generalization and practice")
        - DRAWBACK: symbolic approaches rely on biases to be segmented
    - no direct time dependence
        - DRAWBACK: Algining and scaling time dependet sequences is a difficult task (handling spatial and temporal perturbances is hard)
    - other approaches have considererd modeling the intrinsic dynamics of motion
        - BENEFIT: does not depent on explicit time variable
        - BENEFIT: can be modulated in unseen regions
        - DRAWBACK: require high number of states and smooting procedure
    - propsed model: HMMs and GMMs
    - GMR models joint probability function of the data (no direct regression like gpr)
    - regression function is then derived from the joint density model
        - ADVANTAGE: input and output components are only specified at the very last step of the process
        - ADVANTAGE: density estimation can be learning in an off-line phase, regression process can be computed very rapidly
        - ADVANTAGE: can handle different sources of missing data: system can consider any combination of input/output mappings

## Approach

- a skill is demonstrated to the robot in slightly different situations
- demonstration $m \in \{1, ..., M\}$ consists of a set of Trajectories $T_m$
- Trajectory $T_m$ consists of $d$-dimensional joint positions $x$ and velocities $\dot{x}$ 

$$D = \{\{(x_t, \dot{x}_t)\}_{t=1}^T\}_{m=1}^M$$

- joint distribution $\mathcal{P}(x, \dot{x})$ is encoded in a continuous HMM of K states.
- output distribution of each state is represented by a gaussian which encodes local variation and correlation information
- Parameters of HMM:

$$\{\Pi, a, \mu, \Sigma\}$$

- learned using Baum-Welch Algorithm(variant of the expectation maximization algorithm)
- Input and output components of HMM in each state $s_i$:

$$\mu_i = \left[\begin{array}{c}\mu_i^x \\ \mu_i^{\dot{x}}\end{array}\right] \text{, } \Sigma_i = \left[\begin{array}{cc}\Sigma_i^x & \Sigma_i^{x\dot{x}}\\ \Sigma_i^{\dot{x}x}& \Sigma_i^{\dot{x}}\end{array}\right] $$

- given the current position command, a desired velocity command is estimated using gaussian mixture regression

$$\hat{\dot{x}} = \sum_{i=1}^K h_i(x) [ \mu_i^{\dot{x}} + \Sigma_i^{\dot{x}x}(\Sigma_i^x)^{-1} (x - \mu_i^x)]$$

- where $h_i(x)$ is used to encode the sequential information encapsulated in the HMM:

$$h_i(x_t) = \frac{(\sum_{j=1}^K h_j(x_{t-1}) a_{ji}) \mathcal{N}(x_t; \mu_i^x, \Sigma_i^x)}{\sum_{k=1}^K [(\sum_{j=1}^K h_j(x_{t-1}) a_{jk})\mathcal{N}(x_t; \mu_k^x, \Sigma_k^x)]}$$

- since reproduction is unstable in regions that have not been covered during the demonstration, a secondary term has to be added:

$$\hat{x} = \sum_{i=1}^K h_i(x) [ \mu_i^{x} + \Sigma_i^{x\dot{x}}(\Sigma_i^\dot{x})^{-1} (\dot{x} - \mu_i^\dot{x})]$$


$$\ddot{x} = (\hat{\dot{x}} - \dot{x}) \kappa^{\mathcal{V}} + (\hat{x}-x)\kappa^{\mathcal{P}} $$

- show plots

- the first term allows the robot to follow the demonstrate motion profile, the second term keeps the robot from departing from a known situation and forces it to com back into the subspace of demonstrations

- may lead to oscilations

- use adaptive gains: proportional gain should decrease when the system is close to the demonstrated trajectories

- adaptive gains allow the controller to focus on the other constraints of the task

## Metrics

- M1: RMS error along the motion w.r.t to the demonstration dataset  M_1

- M2: RMS error after DTW; spatial information is prioritized here M_2; the metric compares the path followed by the robot instead of the exact trajectory

- M3: Norm of jerk; derivative of acceleration is a good candidate to evaluate the smoothness of human motion

- M4: Computation time of learning process

- M5: Retrieval duration


## Comparison

- HMM vs TMGR (Time dependent gaussian mixture regression): 

- M1 & M2: all methods perform well, HMM performs well with a small number of states

- M3: HMM is a little bit jerky

- M4: training time is less important than reproduction time

- M5: LWR not competitable; linear dependencs in the number of states

- when dimensionality is low, the difficulty is to correctly handle the crossing points that can appear when randomly generating trajectories



In [ ]:
def plan_demo(pos1, pos2, pos3, start=q_home):
    # first generate target frames
    if "ball1" not in C.getFrameNames():
        C.addObject(name="ball1", shape=ry.ST.sphere, size=[0.01], pos=pos1, color=[0, 0, 1], parent="base_footprint")
        C.addObject(name="ball2", shape=ry.ST.sphere, size=[0.01], pos=pos2, color=[0, 1, 0], parent="base_footprint")
        C.addObject(name="ball3", shape=ry.ST.sphere, size=[0.01], pos=pos3, color=[1, 0, 0], parent="base_footprint")
    C.frame("ball1").setPosition(pos1)
    C.frame("ball2").setPosition(pos2)
    C.frame("ball3").setPosition(pos3)
    path_planner = C.komo_path(1, 300, time_per_phase, False)
    # generate some kind of random start position
    noise = np.random.normal(0, 0.1, start.shape[0]-2)
    # don't change gripper values
    start[0:-2] += noise
    B.moveHard(start)
    B.sync(C)
    path_planner.setConfigurations(C)
    path_planner.clearObjectives()
    path_planner.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=["baxterR", "ball1"], time=[0.3])
    path_planner.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYZ, frames=["baxterR", "ball1"], target=[0], time=[0.3])
    path_planner.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=["baxterR", "ball2"], time=[0.6])
    path_planner.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYZ, frames=["baxterR", "ball2"], target=[0], time=[0.6])
    path_planner.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=["baxterR", "ball3"], time=[1.])
    path_planner.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYZ, frames=["baxterR", "ball3"], target=[0], time=[1])
    path_planner.addObjective(type=ry.OT.sos, feature=ry.FS.qItself, time=[0, 1], target=start)
    path_planner.addObjective(type=ry.OT.sos, feature=ry.FS.qItself, time=[0,  1.], order=1)
    path_planner.optimize(False)
    t = path_planner.getT()
    demo = []
    q_old = np.zeros((7,))
    for name in right_joints:
        q_old[mapping[name]['ros']] = start[mapping[name]['rai']]
    q_ros = np.zeros((7,))
    path = []
    for i in range(t):
        C.setFrameState(path_planner.getConfiguration(i))
        q_new = C.getJointState()
        path += [q_new]
        for name in right_joints:
            q_ros[mapping[name]['ros']] = q_new[mapping[name]['rai']]
        q_dot = q_ros - q_old
        demo.append(np.concatenate([q_old, q_dot]))
        q_old = q_ros
    demo.append(np.concatenate([q_ros, np.zeros((7,))]))
    B.move(path, [time_per_phase/steps_per_phase * i for i in range(t)], False)
    return np.array(demo)

In [ ]:
demos = []
for i in range(3):
    demos.append(plan_demo([1.0, 0.3, 1], [0.0, 0.6, 0.5], [0.5, 0.4, 1.3]))

In [ ]:
# directly setting joint velocities is not standard with the rai framework; playing around with the komo optimizer
def model_step_using_opt(t, kappa_p=0.1, kappa_v=0.1, model=model):
    B.sync(C)
    ik.setConfigurations(C)
    q_rai, q_dot = C.getJointState_qdot()
    q = np.zeros((7, ))
    for name in right_joints:
        q[mapping[name]['ros']] = q_rai[mapping[name]['rai']]
    q_dot = model.predict_qdot(q, t)
    q_new = model.predict_q(q_dot, q, t)
    q_dot_rai = np.zeros((17,))
    for name in right_joints:
        q_rai[mapping[name]['rai']] = q_new[mapping[name]['ros']]
        q_dot_rai[mapping[name]['rai']] = q_dot[mapping[name]['ros']]
    ik.clearObjectives()
    ik.addObjective(feature=ry.FS.qItself, order=0, target=q_rai, type=ry.OT.eq, scale=[1])
    ik.addObjective(feature=ry.FS.qItself, order=1, target=q_dot_rai, type=ry.OT.eq, scale=[1])
    ik.optimize(False)
    C.setFrameState(ik.getConfiguration(0))
    q_rai = C.getJointState()
    B.move([q_rai], [1], True)
    return np.concatenate([q_new, q_dot])

# get velocity and new joint configuration from HMM-model, directly set joint configurations
def model_step(t, model=model):
    B.sync(C)
    q_rai = C.getJointState()
    q = np.zeros((7, ))
    for name in right_joints:
        q[mapping[name]['ros']] = q_rai[mapping[name]['rai']]
    q_dot = model.predict_qdot(q, t)
    q_new = model.predict_q(q_dot, q, t)
    for name in right_joints:
        q_rai[mapping[name]['rai']] = q_new[mapping[name]['ros']]
    B.move([q_rai], [0.1], True)
    return np.concatenate([q_new, q_dot])

def run_model(n_steps, ik=False, model=model):
    B.moveHard(q_home)
    C.setJointState(q_home)
    trajectory = []
    for i in range(n_steps):
        time.sleep(0.1)
        if ik:
            trajectory += [model_step_using_opt(i)]
        else:
            trajectory += [model_step(i)]
    return trajectory

def run_demo(demo):
    q_rai = np.zeros((17,))
    for i, qqdot in enumerate(demo):
        for name in right_joints:
            q_rai[mapping[name]['rai']] = qqdot[mapping[name]['ros']]
        time.sleep(1)
        B.move([q_rai], [0.1], True)